In [1]:
import numpy as np
import scipy
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, Conv3D
from keras.optimizers import SGD
from keras import backend as K
#K.set_image_dim_ordering('th')
from keras.utils import to_categorical
#from sklearn.cross_validation import StratifiedKFold

In [2]:
PATH = os.getcwd()
print (PATH)

C:\Users\tasni\Desktop\STUDY\GL4\SEM2\ProjectTI\Spectral_Images_Processing


In [3]:
# Load the Global values (windowSize, numPCAcomponents, testRatio) from the text file global_variables.txt
myFile = open('global_variables.txt', 'r') 
file = myFile.readlines()[:]


for line in file:

    if line[0:3] == "win":

        ds = line.find('=')
        windowSize = int(line[ds+1:-1],10)

    elif line[0:3] == "num":

        ds = line.find('=')
        numPCAcomponents = int(line[ds+2:-1],10)

    else:

        ds = line.find('=')
        testRatio = float(line[ds+1:])


In [4]:
# Global Variables
#windowSize = 5
#numPCAcomponents = 30
#testRatio = 0.25

# Load Training Dataset

In [5]:
X_train = np.load("X_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio)  + ".npy")

y_train = np.load("y_trainPatches_" + str(windowSize) + "PCA" + str(numPCAcomponents) + "testRatio" + str(testRatio) + ".npy")

In [6]:
# Reshape into (numberofsamples, channels, height, width)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[3], X_train.shape[1], X_train.shape[2]))

In [7]:
# convert class labels to on-hot encoding
y_train = to_categorical(y_train)

In [8]:
import tensorflow as tf

# Define the input shape based on your windowSize and numPCAcomponents
input_shape = (windowSize, windowSize, numPCAcomponents, 1)

print(input_shape)

(15, 15, 30, 1)


In [9]:


# Define the model
model = Sequential()

model.add(Conv3D(16, (3, 3, numPCAcomponents),padding='same', input_shape=input_shape))
model.add(Conv3D(32, (3, 3, 5),padding='same'))  # Adjust the kernel size as needed
model.add(Flatten())
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(512, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(16, activation="softmax"))

model.summary()




C:\Users\tasni\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                      │ (None, 15, 15, 30, 16)      │           4,336 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv3d_1 (Conv3D)                    │ (None, 15, 15, 30, 32)      │          23,072 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 216000)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │      55,296,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 16)                  │           8,208 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 55,463,456 (211.58 MB)

 Trainable params: 55,463,456 (211.58 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
sgd = SGD(learning_rate=0.0001, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [11]:
history= model.fit(X_train, y_train, batch_size=32, epochs=15)

Epoch 1/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 362s 388ms/step - accuracy: 0.5236 - loss: 1.7351
Epoch 2/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 364s 392ms/step - accuracy: 0.9110 - loss: 0.3110
Epoch 3/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 360s 388ms/step - accuracy: 0.9575 - loss: 0.1490
Epoch 4/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 371s 400ms/step - accuracy: 0.9749 - loss: 0.0925
Epoch 5/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 385s 414ms/step - accuracy: 0.9837 - loss: 0.0611
Epoch 6/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 384s 414ms/step - accuracy: 0.9886 - loss: 0.0432
Epoch 7/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 395s 426ms/step - accuracy: 0.9902 - loss: 0.0365
Epoch 8/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 359s 387ms/step - accuracy: 0.9911 - loss: 0.0313
Epoch 9/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 359s 387ms/step - accuracy: 0.9946 - loss: 0.0237
Epoch 10/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 361s 389ms/step - accuracy: 0.9952 - loss: 0.0203
Epoch 11/15
928/928 ━━━━━━━━━━━━━━━━━━━━ 360s 388ms/step - accuracy: 0.9958 - loss: 0.0184
Epoch 12

In [12]:
# Evaluate the model on the training data
train_loss, train_accuracy = model.evaluate(X_train, y_train)

# Print or use the metrics as needed
print("Training Loss:", train_loss)
print("Training Accuracy:", train_accuracy)


928/928 ━━━━━━━━━━━━━━━━━━━━ 99s 107ms/step - accuracy: 1.0000 - loss: 1.5044e-04
Training Loss: 0.000151686355820857
Training Accuracy: 1.0


In [13]:
import h5py
from keras.models import load_model

In [14]:
model.save('my_model2' + str(windowSize) + 'PCA' + str(numPCAcomponents) + "testRatio" + str(testRatio)+'.keras')